# Urban/Rural Analysis of CT Hospital Benefits

This notebook demonstrates how to analyze benefit areas by their rural/urban characteristics using the GHS-SMOD (Global Human Settlement Model - Settlement Model) raster data.

## Overview

The GHS-SMOD raster classifies areas by degree of urbanization using UN standards:
- **Urban codes**: {30, 23, 22, 21} - Cities, towns, suburban areas
- **Rural codes**: {13, 12, 11} - Rural areas with different densities
- **Other codes**: Water bodies, very low density areas, etc.

## Requirements

1. **Benefit analysis results**: Previously generated using `geostroke.benefit` functions
2. **GHS-SMOD raster**: Download from [GHS-SMOD webpage](https://ghsl.jrc.ec.europa.eu/ghs_smod2023.php)
   - File: `GHS_SMOD_E2025_GLOBE_R2023A_54009_1000_V2_0.tif` (or similar)
   - Place in `raw_data/` directory


In [2]:
# ---- 1. Install what you might still need ----
# pip install geopandas rasterio pandas pyproj shapely

import sys
from pathlib import Path

# Set working directory to project root
import os
os.chdir('..')  # Go

import geostroke as gs
import geopandas as gpd
import pandas as pd
import pickle
import numpy as np
from pathlib import Path

print(f"✅ Loaded geostroke version {gs.__version__}")
print(f"📁 Project root: {gs.config.ROOT}")
print(f"📁 Data directory: {gs.config.DATA_DIR}")


✅ Loaded geostroke version 1.0.0
📁 Project root: /Users/larsmasanneck/Library/CloudStorage/OneDrive-Personal/Dokumente/Research Basics and Coding Projects/Data Projects/Marc Pawlitzki/GeoStroke
📁 Data directory: /Users/larsmasanneck/Library/CloudStorage/OneDrive-Personal/Dokumente/Research Basics and Coding Projects/Data Projects/Marc Pawlitzki/GeoStroke/raw_data


## Configuration

**Update these paths for your specific files:**

In [3]:
# ---- 2. Point to your inputs ----

BENEFIT_FILE = gs.config.DATA_DIR / "benefit_cache" / "benefit_analysis_e1f09274ca3df68bc9344be034cc005f.pkl"  # <-- change to your actual file


# GHS-SMOD raster path (adjust for your file)
SMOD_RASTER = gs.config.DATA_DIR / "GHS_SMOD_E2025_GLOBE_R2023A_54009_1000_V2_0.tif"

# Check if files exist
print(f"\n🔍 File checks:")
print(f"   Benefit file exists: {'✅' if BENEFIT_FILE and BENEFIT_FILE.exists() else '❌'}")
print(f"   SMOD raster exists: {'✅' if SMOD_RASTER.exists() else '❌'}")

if BENEFIT_FILE:
    print(f"   Benefit file: {BENEFIT_FILE}")
if SMOD_RASTER.exists():
    print(f"   SMOD raster: {SMOD_RASTER}")
else:
    print(f"   ❌ SMOD raster not found at: {SMOD_RASTER}")
    print(f"   📥 Please download from: https://ghsl.jrc.ec.europa.eu/ghs_smod2023.php")



🔍 File checks:
   Benefit file exists: ✅
   SMOD raster exists: ✅
   Benefit file: /Users/larsmasanneck/Library/CloudStorage/OneDrive-Personal/Dokumente/Research Basics and Coding Projects/Data Projects/Marc Pawlitzki/GeoStroke/raw_data/benefit_cache/benefit_analysis_e1f09274ca3df68bc9344be034cc005f.pkl
   SMOD raster: /Users/larsmasanneck/Library/CloudStorage/OneDrive-Personal/Dokumente/Research Basics and Coding Projects/Data Projects/Marc Pawlitzki/GeoStroke/raw_data/GHS_SMOD_E2025_GLOBE_R2023A_54009_1000_V2_0.tif


## Alternative: Generate Benefit Analysis

If you don't have cached benefit results, run this cell to generate them:


In [4]:
# ---- Alternative: Generate benefit analysis if needed ----

if not BENEFIT_FILE or not BENEFIT_FILE.exists():
    print("🔄 Generating benefit analysis (this may take several minutes)...")
    
    # Quick benefit analysis with coarser resolution for demo
    benefit_gdf = gs.benefit.calculate_time_benefits_parallel(
        ct_penalty=0.0,
        benefit_threshold=10.0,
        grid_resolution=0.01,  # Coarser resolution for faster processing
        time_bins=[5,10,15,20,25, 30,35,40, 45,50,55, 60],  # Fewer time bins
        max_workers=4
    )
    
    print(f"✅ Generated benefit analysis with {len(benefit_gdf):,} points")
    
else:
    print("✅ Benefit file exists, will load from cache")
    benefit_gdf = None  # Will be loaded in next cell


✅ Benefit file exists, will load from cache


## Load Benefit Analysis Data


In [5]:
# ---- 3. Load the benefit GeoDataFrame ----

if benefit_gdf is None:  # Not generated in previous cell
    if BENEFIT_FILE and BENEFIT_FILE.exists():
        print(f"📥 Loading benefit analysis from: {BENEFIT_FILE.name}")
        
        if BENEFIT_FILE.suffix == ".pkl":
            with BENEFIT_FILE.open("rb") as f:
                cached_data = pickle.load(f)
            
            # Handle both old and new cache formats
            if isinstance(cached_data, dict) and "data" in cached_data:
                # New format with metadata
                benefit_gdf = gpd.GeoDataFrame(cached_data["data"], crs="EPSG:4326")
                if "params" in cached_data:
                    print(f"   📋 Analysis parameters: {cached_data['params']}")
            else:
                # Old format - direct data
                benefit_gdf = gpd.GeoDataFrame(cached_data, crs="EPSG:4326")
        else:
            # GeoPackage or other format
            benefit_gdf = gpd.read_file(BENEFIT_FILE)
            
        print(f"✅ Loaded {len(benefit_gdf):,} benefit points")
        
    else:
        raise FileNotFoundError("No benefit file available. Please run benefit analysis first.")

# Display basic info about the benefit data
print(f"\n📊 Benefit Data Summary:")
print(f"   Total points: {len(benefit_gdf):,}")
print(f"   Coordinate system: {benefit_gdf.crs}")
print(f"   Columns: {list(benefit_gdf.columns)}")
print(f"\n   Benefit categories:")
for category, count in benefit_gdf['benefit_category'].value_counts().items():
    print(f"     {category}: {count:,} points")


📥 Loading benefit analysis from: benefit_analysis_e1f09274ca3df68bc9344be034cc005f.pkl
   📋 Analysis parameters: {'ct_suffix': '_all_CTs', 'stroke_suffix': '', 'time_bins': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60], 'ct_penalty': 0, 'benefit_threshold': 10.0, 'grid_resolution': 0.01, 'bounds': None}
✅ Loaded 510,229 benefit points

📊 Benefit Data Summary:
   Total points: 510,229
   Coordinate system: EPSG:4326
   Columns: ['geometry', 'ct_time', 'stroke_time', 'time_benefit', 'benefit_category']

   Benefit categories:
     Neither reachable within 60 min: 260,954 points
     Low (10-20 min): 117,029 points
     Medium (20-30 min): 72,853 points
     High (30+ min): 37,724 points
     Only CT reachable within 60 min: 21,106 points
     Likely irrelevant (<10 min): 563 points


## Urban/Rural Annotation

Now we'll add urban/rural labels to each benefit point using the GHS-SMOD raster:


In [6]:
# ---- 4. Annotate every point with rural/urban classification (FIXED to include full complement) ----
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

# Sanity check
if not SMOD_RASTER.exists():
    raise FileNotFoundError(f"SMOD raster not found at {SMOD_RASTER}")

# Germany outline & polygon
GERMANY = gs.data.load_germany_outline()
GERMANY_POLY = GERMANY.geometry.iloc[0]

# Infer grid resolution from the cached benefit points (fallback 0.01°)
xs_u = np.sort(benefit_gdf.geometry.x.unique())
ys_u = np.sort(benefit_gdf.geometry.y.unique())
try:
    dx = np.median(np.diff(xs_u))
    dy = np.median(np.diff(ys_u))
    GRID_RES = round(float(min(dx, dy)), 5)
    if GRID_RES <= 0 or GRID_RES > 0.05:
        GRID_RES = 0.01
except Exception:
    GRID_RES = 0.01

# Build full Germany grid
minx, miny, maxx, maxy = GERMANY.total_bounds
xs = np.arange(minx, maxx + GRID_RES, GRID_RES)
ys = np.arange(miny, maxy + GRID_RES, GRID_RES)
pts = [Point(x, y) for x in xs for y in ys]
GERMANY_GRID = gpd.GeoDataFrame(
    geometry=[p for p in pts if GERMANY_POLY.contains(p) or GERMANY_POLY.touches(p)],
    crs=GERMANY.crs
)

# Map categories from benefit_gdf to the full grid (no spatial join with points!)
def _ck(p):  # coordinate key
    return (round(p.x, 5), round(p.y, 5))

cat_lookup = dict(zip(
    benefit_gdf.geometry.apply(_ck),
    benefit_gdf['benefit_category']
))

GERMANY_GRID['benefit_category'] = GERMANY_GRID.geometry.apply(
    lambda g: cat_lookup.get(_ck(g), "Likely irrelevant (<10 min)")
)

print(f"Total Germany grid points: {len(GERMANY_GRID):,}")
print("Category counts (points):")
print(GERMANY_GRID['benefit_category'].value_counts())

# Annotate with SMOD
print("🏙️ Annotating grid with SMOD urban/rural classes …")
annotated = gs.urban_rural_annotation.add_smod_labels(GERMANY_GRID, str(SMOD_RASTER))
print("✅ Annotation done")

# Quick check
ur_counts = annotated['urban_rural'].value_counts()
print("Urban/Rural distribution (all points):")
print(ur_counts.apply(lambda v: f"{v:,} ({v/len(annotated)*100:.1f}%)"))

Total Germany grid points: 459,349
Category counts (points):
benefit_category
Likely irrelevant (<10 min)        206068
Low (10-20 min)                    116674
Medium (20-30 min)                  72459
High (30+ min)                      37417
Only CT reachable within 60 min     20162
Neither reachable within 60 min      6569
Name: count, dtype: int64
🏙️ Annotating grid with SMOD urban/rural classes …
✅ Annotation done
Urban/Rural distribution (all points):
urban_rural
rural    412,940 (89.9%)
urban      44,554 (9.7%)
other       1,855 (0.4%)
Name: count, dtype: object


In [7]:
# ---- 5. Urban/Rural summary table (absolute + %) by benefit category ----
print("📋 Generating Urban/Rural Summary by Benefit Category")
print("=" * 70)

CATEGORY_ORDER = [
    "High (30+ min)",
    "Medium (20-30 min)",
    "Low (10-20 min)",
    "Only CT reachable within 60 min",
    "Neither reachable within 60 min",
    "Likely irrelevant (<10 min)",
]

# Counts
cnt = (annotated
       .groupby(['benefit_category', 'urban_rural'])
       .size()
       .rename('n')
       .reset_index())

# Totals per category & percentages
totals = cnt.groupby('benefit_category')['n'].sum().rename('cat_total')
cnt = cnt.merge(totals, on='benefit_category')
cnt['pct_within_cat'] = (cnt['n'] / cnt['cat_total'] * 100).round(1)

# Wide tables
summary_abs = (cnt.pivot_table(index='benefit_category',
                               columns='urban_rural',
                               values='n',
                               fill_value=0)
                 .reindex(CATEGORY_ORDER))
summary_pct = (cnt.pivot_table(index='benefit_category',
                               columns='urban_rural',
                               values='pct_within_cat',
                               fill_value=0)
                 .reindex(CATEGORY_ORDER))

pd.set_option('display.width', None)
print("\n👥 Absolute counts:")
print(summary_abs.applymap(lambda x: f"{x:,}"))

print("\n📊 Percent within each category:")
print(summary_pct)

# Save
summary_abs.to_csv("urban_rural_by_category_abs.csv")
summary_pct.to_csv("urban_rural_by_category_pct.csv")
print("\n💾 Saved CSVs: urban_rural_by_category_abs.csv, urban_rural_by_category_pct.csv")

# Markdown export
print("\n📋 Markdown (percent table):")
print(summary_pct.to_markdown())

📋 Generating Urban/Rural Summary by Benefit Category

👥 Absolute counts:
urban_rural                      other      rural     urban
benefit_category                                           
High (30+ min)                    47.0   34,966.0   2,404.0
Medium (20-30 min)                93.0   68,063.0   4,303.0
Low (10-20 min)                  242.0  106,732.0   9,700.0
Only CT reachable within 60 min  164.0   19,760.0     238.0
Neither reachable within 60 min  833.0    5,548.0     188.0
Likely irrelevant (<10 min)      476.0  177,871.0  27,721.0

📊 Percent within each category:
urban_rural                      other  rural  urban
benefit_category                                    
High (30+ min)                     0.1   93.4    6.4
Medium (20-30 min)                 0.1   93.9    5.9
Low (10-20 min)                    0.2   91.5    8.3
Only CT reachable within 60 min    0.8   98.0    1.2
Neither reachable within 60 min   12.7   84.5    2.9
Likely irrelevant (<10 min)        0.2   86

/var/folders/l_/hnp4n5y13k3dbz2xfjjz44p00000gn/T/ipykernel_1951/1045961716.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(summary_abs.applymap(lambda x: f"{x:,}"))


## Summary Analysis

Generate summary tables showing the urban/rural breakdown by benefit category:


In [8]:
# ---- 5. Build the summary percentage table ----

print("📋 Generating Urban/Rural Summary by Benefit Category")
print("=" * 55)

summary = gs.urban_rural_annotation.summary_by_category(annotated)

# Display the table in a nice format
print("\n📊 Summary Table:")
print(summary.to_string())

# Also display as markdown for better formatting
print("\n📋 Markdown format (for easy copying):")
print(summary.to_markdown())

# Export to CSV
output_file = "benefit_urban_rural_summary.csv"
summary.to_csv(output_file)
print(f"\n💾 Summary exported to: {output_file}")


📋 Generating Urban/Rural Summary by Benefit Category

📊 Summary Table:
urban_rural                      other   rural  urban   total  pct_urban  pct_rural  pct_other
benefit_category                                                                              
High (30+ min)                      47   34966   2404   37417        6.4       93.4        0.1
Likely irrelevant (<10 min)        476  177871  27721  206068       13.5       86.3        0.2
Low (10-20 min)                    242  106732   9700  116674        8.3       91.5        0.2
Medium (20-30 min)                  93   68063   4303   72459        5.9       93.9        0.1
Neither reachable within 60 min    833    5548    188    6569        2.9       84.5       12.7
Only CT reachable within 60 min    164   19760    238   20162        1.2       98.0        0.8

📋 Markdown format (for easy copying):
| benefit_category                |   other |   rural |   urban |   total |   pct_urban |   pct_rural |   pct_other |
|:----------

## Key Insights

Generate some key insights from the analysis:


In [9]:
# ---- 6. Generate key insights ----

print("💡 Key Insights from Urban/Rural Analysis")
print("=" * 45)

# Overall urban vs rural distribution
total_urban = summary['urban'].sum() if 'urban' in summary.columns else 0
total_rural = summary['rural'].sum() if 'rural' in summary.columns else 0
total_other = summary['other'].sum() if 'other' in summary.columns else 0
total_points = total_urban + total_rural + total_other

print(f"\n🌍 Overall Distribution:")
print(f"   Urban areas: {total_urban:,} points ({total_urban/total_points*100:.1f}%)")
print(f"   Rural areas: {total_rural:,} points ({total_rural/total_points*100:.1f}%)")
print(f"   Other areas: {total_other:,} points ({total_other/total_points*100:.1f}%)")

# Find which benefit categories are most rural vs urban
print(f"\n🏙️ Most Urban Benefit Categories:")
if 'pct_urban' in summary.columns:
    urban_sorted = summary.sort_values('pct_urban', ascending=False)
    for category in urban_sorted.index[:3]:
        pct = urban_sorted.loc[category, 'pct_urban']
        count = urban_sorted.loc[category, 'urban'] if 'urban' in urban_sorted.columns else 0
        print(f"   {category}: {pct:.1f}% urban ({count:,} points)")

print(f"\n🌾 Most Rural Benefit Categories:")
if 'pct_rural' in summary.columns:
    rural_sorted = summary.sort_values('pct_rural', ascending=False)
    for category in rural_sorted.index[:3]:
        pct = rural_sorted.loc[category, 'pct_rural']
        count = rural_sorted.loc[category, 'rural'] if 'rural' in rural_sorted.columns else 0
        print(f"   {category}: {pct:.1f}% rural ({count:,} points)")

# Healthcare desert analysis
if "Neither reachable within 60 min" in summary.index:
    desert_row = summary.loc["Neither reachable within 60 min"]
    print(f"\n🏜️ Healthcare Desert Analysis:")
    print(f"   Total healthcare desert areas: {desert_row['total']:,} points")
    if 'pct_rural' in desert_row:
        print(f"   Rural healthcare deserts: {desert_row['pct_rural']:.1f}%")
    if 'pct_urban' in desert_row:
        print(f"   Urban healthcare deserts: {desert_row['pct_urban']:.1f}%")

print(f"\n✅ Analysis complete! Check the exported CSV files for detailed results.")


💡 Key Insights from Urban/Rural Analysis

🌍 Overall Distribution:
   Urban areas: 44,554 points (9.7%)
   Rural areas: 412,940 points (89.9%)
   Other areas: 1,855 points (0.4%)

🏙️ Most Urban Benefit Categories:
   Likely irrelevant (<10 min): 13.5% urban (27,721 points)
   Low (10-20 min): 8.3% urban (9,700 points)
   High (30+ min): 6.4% urban (2,404 points)

🌾 Most Rural Benefit Categories:
   Only CT reachable within 60 min: 98.0% rural (19,760 points)
   Medium (20-30 min): 93.9% rural (68,063 points)
   High (30+ min): 93.4% rural (34,966 points)

🏜️ Healthcare Desert Analysis:
   Total healthcare desert areas: 6,569.0 points
   Rural healthcare deserts: 84.5%
   Urban healthcare deserts: 2.9%

✅ Analysis complete! Check the exported CSV files for detailed results.


## Isochrone Coverage Analysis

In addition to analyzing specific benefit points, we can examine the raw isochrone coverage areas themselves. This provides insights into the spatial extent of healthcare accessibility across urban and rural areas for different time thresholds.

### What This Analysis Shows

- **Coverage patterns** - How much area is covered by each time bin
- **Urban vs rural accessibility** - Whether urban or rural areas dominate coverage zones  
- **Facility type comparison** - Differences between stroke units and CT hospitals
- **Time-distance relationships** - How coverage expands with time thresholds


In [10]:
# ---- 7. Load and Process Isochrone Data ----

def load_isochrones_as_gdf(suffix: str, facility_type: str, time_bins: list = None) -> gpd.GeoDataFrame:
    """Load isochrone polygons and convert to GeoDataFrame with metadata."""
    if time_bins is None:
        time_bins = [15, 30, 45, 60]  # Focus on key time bands for visualization
    
    # Load facility data to get facility information
    if suffix == "_all_CTs":
        facilities_df = gs.data.load_hospitals_ct()
    elif suffix == "_extended_stroke":
        try:
            facilities_df = gs.data.load_extended_stroke_units()
        except FileNotFoundError as e:
            print(f"⚠️  Extended stroke units not available: {e}")
            print(f"   Please run additional_stroke_centers.ipynb to generate the extended dataset.")
            return gpd.GeoDataFrame(columns=['geometry', 'time_bin', 'facility_type', 'facility_id', 
                                           'facility_name', 'facility_lat', 'facility_lon', 'area_km2'], 
                                   crs="EPSG:4326")
    else:
        facilities_df = gs.data.load_stroke_units()
    
    rows = []
    
    for time_bin in time_bins:
        cache_path = gs.config.DATA_DIR / f"poly{time_bin}{suffix}.pkl"
        
        if cache_path.exists():
            with open(cache_path, "rb") as f:
                polygons = pickle.load(f)
            
            print(f"📦 Loaded {len(polygons)} polygons for {facility_type} {time_bin}min")
            
            # Create rows for each polygon with metadata
            for i, polygon in enumerate(polygons):
                if polygon and not polygon.is_empty:  # Skip empty/invalid polygons
                    # Get facility info if available
                    facility_name = "Unknown"
                    facility_lat = None
                    facility_lon = None
                    
                    if i < len(facilities_df):
                        facility_row = facilities_df.iloc[i]
                        facility_name = facility_row.get('name', f'Facility_{i}')
                        facility_lat = facility_row.get('latitude')
                        facility_lon = facility_row.get('longitude')
                    
                    rows.append({
                        'geometry': polygon,
                        'time_bin': time_bin,
                        'facility_type': facility_type,
                        'facility_id': i,
                        'facility_name': facility_name,
                        'facility_lat': facility_lat,
                        'facility_lon': facility_lon,
                        'area_km2': polygon.area * 111**2,  # Rough conversion to km²
                    })
        else:
            print(f"⚠️  Missing isochrone file: {cache_path}")
    
    if rows:
        return gpd.GeoDataFrame(rows, crs="EPSG:4326")
    else:
        return gpd.GeoDataFrame(columns=['geometry', 'time_bin', 'facility_type', 'facility_id', 
                                       'facility_name', 'facility_lat', 'facility_lon', 'area_km2'], 
                               crs="EPSG:4326")

print("🔄 Loading isochrone data for analysis...")
print("   This may take a moment as we process thousands of polygons...")

# Load isochrones for both facility types
stroke_isochrones = load_isochrones_as_gdf("", "Stroke Units")
ct_isochrones = load_isochrones_as_gdf("_all_CTs", "CT Hospitals")
extended_stroke_isochrones = load_isochrones_as_gdf("_extended_stroke", "Extended Stroke Units")

# Combine both datasets
combined_data = pd.concat([stroke_isochrones, ct_isochrones, extended_stroke_isochrones], ignore_index=True)

all_isochrones = gpd.GeoDataFrame(combined_data, crs="EPSG:4326")

print(f"\n✅ Loaded isochrone data:")
print(f"   Stroke units: {len(stroke_isochrones):,} isochrone polygons")
print(f"   CT hospitals: {len(ct_isochrones):,} isochrone polygons")
print(f"   Extended stroke units: {len(extended_stroke_isochrones):,} isochrone polygons")
print(f"   Total: {len(all_isochrones):,} isochrone polygons")


# Display basic statistics
print(f"\n📊 Isochrone Coverage Summary:")
for facility_type in all_isochrones['facility_type'].unique():
    subset = all_isochrones[all_isochrones['facility_type'] == facility_type]
    print(f"   {facility_type}:")
    for time_bin in sorted(subset['time_bin'].unique()):
        time_subset = subset[subset['time_bin'] == time_bin]
        total_area = time_subset['area_km2'].sum()
        avg_area = time_subset['area_km2'].mean()
        print(f"     {time_bin} min: {len(time_subset):,} polygons, {total_area:,.0f} km² total, {avg_area:.1f} km² avg")


🔄 Loading isochrone data for analysis...
   This may take a moment as we process thousands of polygons...
📦 Loaded 349 polygons for Stroke Units 15min
📦 Loaded 349 polygons for Stroke Units 30min
📦 Loaded 349 polygons for Stroke Units 45min
📦 Loaded 349 polygons for Stroke Units 60min
📦 Loaded 1475 polygons for CT Hospitals 15min
📦 Loaded 1475 polygons for CT Hospitals 30min
📦 Loaded 1475 polygons for CT Hospitals 45min
📦 Loaded 1475 polygons for CT Hospitals 60min
📦 Loaded 463 polygons for Extended Stroke Units 15min
📦 Loaded 463 polygons for Extended Stroke Units 30min
📦 Loaded 463 polygons for Extended Stroke Units 45min
📦 Loaded 463 polygons for Extended Stroke Units 60min

✅ Loaded isochrone data:
   Stroke units: 1,396 isochrone polygons
   CT hospitals: 5,900 isochrone polygons
   Extended stroke units: 1,852 isochrone polygons
   Total: 9,148 isochrone polygons

📊 Isochrone Coverage Summary:
   Stroke Units:
     15 min: 349 polygons, 75,255 km² total, 215.6 km² avg
     30 min

/Users/larsmasanneck/Library/CloudStorage/OneDrive-Personal/Dokumente/Research Basics and Coding Projects/Data Projects/Marc Pawlitzki/GeoStroke/geostroke/data.py:79: RuntimeWarning: stroke_units_extended_geocoded.csv found in project root; consider moving it to /Users/larsmasanneck/Library/CloudStorage/OneDrive-Personal/Dokumente/Research Basics and Coding Projects/Data Projects/Marc Pawlitzki/GeoStroke/raw_data
  warnings.warn(


In [11]:
"""
Union isochrones per *facility type* and *time bin* – **Version 3 (debug)**
=======================================================================
This revision fixes the >100 % artefacts you saw:

* **all_touched=False** – we now count only pixels whose *centre* lies in
the polygon, eliminating boundary over-counting.
* **Pixel → km² conversion** – absolute urban/rural areas are derived from
  `pixel_count × pixel_area`, so the km² sums line up with the polygon
  surface.
* **Debug block** – prints a side-by-side comparison of polygon area vs
  raster-derived area so you can spot any remaining discrepancy.

Drop this cell *after* you create `all_isochrones` (immediately after cell 2).
It leaves the `unions` GeoDataFrame in memory and writes two CSVs mirroring
your earlier grid analysis.
"""

from pathlib import Path

import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rasterstats as rs

EA_CRS = "EPSG:6933"   # Cylindrical Equal-Area (m)
EA_RES = 1000           # 1 km × 1 km grid ⇒ 1 pixel ≈ 1 km²
PIXEL_AREA_KM2 = (EA_RES ** 2) / 1e6

#############################################
# 0. Re-project SMOD raster to equal-area CRS
#############################################
SMOD_EA = Path(str(SMOD_RASTER).replace(".tif", "_EA.tif"))
if not SMOD_EA.exists():
    print("🔄 Re-projecting SMOD raster to equal-area …")
    with rasterio.open(SMOD_RASTER) as src:
        transform, width, height = calculate_default_transform(
            src.crs, EA_CRS, src.width, src.height,
            *src.bounds, resolution=EA_RES
        )
        meta = src.meta.copy()
        meta.update({"crs": EA_CRS, "transform": transform,
                     "width": width, "height": height})
        with rasterio.open(SMOD_EA, "w", **meta) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    rasterio.band(src, i), rasterio.band(dst, i),
                    src_transform=src.transform, src_crs=src.crs,
                    dst_transform=transform, dst_crs=EA_CRS,
                    resampling=Resampling.nearest,
                )
    print(f"   → Saved {SMOD_EA.name}")
else:
    print(f"📦 Using cached equal-area SMOD raster: {SMOD_EA.name}")

#####################
# 1. Union polygons #
#####################
print("🧮 Dissolving isochrones by facility type + time band …")
iso_eq = all_isochrones.to_crs(EA_CRS)

after_union = (iso_eq.dissolve(by=["facility_type", "time_bin"])  # GeoSeries
                        .reset_index())
after_union["area_km2"] = after_union.geometry.area / 1e6  # m² → km²
print(f"   {len(after_union):,} union polygons ready")

############################################
# 2. Zonal statistics on equal-area raster  #
############################################
print("🏙️  Zonal statistics … (centre-pixel test)")

URBAN_CODES = {30, 23,22,21}          # GHSL-SMOD urban
RURAL_CODES = {13,12,11}  # rural / peri-urban
OTHER_CODES = {10}

zs = rs.zonal_stats(
    after_union.geometry, str(SMOD_EA), categorical=True, nodata=0,
    all_touched=False  # centre of pixel must fall within polygon
)

after_union["urban_cells"] = [sum(z.get(c, 0) for c in URBAN_CODES) for z in zs]
after_union["rural_cells"] = [sum(z.get(c, 0) for c in RURAL_CODES) for z in zs]
after_union["other_cells"] = [sum(z.get(c, 0) for c in OTHER_CODES) for z in zs]
after_union["total_cells"] = after_union["urban_cells"] + after_union["rural_cells"] + after_union["other_cells"]

# Convert counts to km²
for col in ["urban_cells", "rural_cells", "other_cells", "total_cells"]:
    after_union[col.replace("cells", "km2")] = after_union[col] * PIXEL_AREA_KM2

# Percentages
after_union["pct_urban"] = (
    after_union["urban_km2"] / after_union["total_km2"] * 100
).round(1)
after_union["pct_rural"] = (
    after_union["rural_km2"] / after_union["total_km2"] * 100
).round(1)
after_union["pct_other"] = (
    after_union["other_km2"] / after_union["total_km2"] * 100
).round(1)

##########################
# 3. Debug consistency 🚦 #
##########################
after_union["area_diff_pct"] = ((after_union["total_km2"] - after_union["area_km2"]) /
                                 after_union["area_km2"] * 100).round(1)

print("\n🔎 Polygon area vs raster-derived area (km²):")
print(after_union[["facility_type", "time_bin", "area_km2", "total_km2", "area_diff_pct"]]
      .to_string(index=False, formatters={
          "area_km2": "{:.0f}".format,
          "total_km2": "{:.0f}".format,
          "area_diff_pct": "{:+.1f}".format,
      }))

###################################################
# 4. Pretty print & write tidy CSVs               #
###################################################
print("\n📋 Union coverage – absolute areas (km²):")
abs_cols = ["facility_type", "time_bin", "total_km2", "urban_km2", "rural_km2", "other_km2"]
print(after_union[abs_cols].to_string(index=False, formatters={
    "total_km2": "{:.0f}".format,
    "urban_km2": "{:.0f}".format,
    "rural_km2": "{:.0f}".format,
    "other_km2": "{:.0f}".format,
}))

print("\n📊 Percent urban/rural within each union polygon:")
print(after_union[["facility_type", "time_bin", "pct_urban", "pct_rural", "pct_other"]]
      .to_string(index=False))

print("\n💾 Writing CSVs …")
after_union[abs_cols].to_csv("union_isochrones_area_abs.csv", index=False)
after_union[["facility_type", "time_bin", "pct_urban", "pct_rural", "pct_other"]].to_csv("union_isochrones_area_pct.csv", index=False)
print("   • union_isochrones_area_abs.csv (absolute km²)\n   • union_isochrones_area_pct.csv (percentages)")

###############################
# 5. Keep result in workspace #
###############################
# A convenient alias for downstream analysis
unions = after_union.copy()

📦 Using cached equal-area SMOD raster: GHS_SMOD_E2025_GLOBE_R2023A_54009_1000_V2_0_EA.tif
🧮 Dissolving isochrones by facility type + time band …
   12 union polygons ready
🏙️  Zonal statistics … (centre-pixel test)

🔎 Polygon area vs raster-derived area (km²):
        facility_type  time_bin area_km2 total_km2 area_diff_pct
         CT Hospitals        15   130656    130696          +0.0
         CT Hospitals        30   320190    320046          -0.0
         CT Hospitals        45   352128    352069          -0.0
         CT Hospitals        60   354706    354669          -0.0
Extended Stroke Units        15    52400     52364          -0.1
Extended Stroke Units        30   226113    226106          -0.0
Extended Stroke Units        45   320398    320313          -0.0
Extended Stroke Units        60   347818    347795          -0.0
         Stroke Units        15    41404     41372          -0.1
         Stroke Units        30   195443    195465          +0.0
         Stroke Units   

In [12]:
# ---- 8. Calculate Total Germany Area Breakdown by Urban/Rural ----

print("🇩🇪 Calculating Germany's total area breakdown by urban/rural categories")
print("=" * 70)

# Load Germany boundary from the detailed geojson file
germany_boundary_path = Path("shp/germany-detailed-boundary_917.geojson")

if not germany_boundary_path.exists():
    print(f"❌ Germany boundary file not found: {germany_boundary_path}")
    print("   Please ensure the file exists in the shp/ directory")
else:
    # Load Germany boundary
    germany_gdf = gpd.read_file(germany_boundary_path)
    print(f"📦 Loaded Germany boundary from {germany_boundary_path.name}")
    
    # Convert to equal-area projection for accurate area calculations
    germany_ea = germany_gdf.to_crs(EA_CRS)
    
    # Calculate polygon area in km²
    germany_polygon_area = germany_ea.geometry.area.sum() / 1e6  # m² to km²
    
    print(f"📐 Germany polygon area: {germany_polygon_area:,.0f} km²")
    
    # Perform zonal statistics on the entire Germany boundary
    print("🏙️ Calculating urban/rural breakdown using SMOD raster...")
    
    # Use the same SMOD raster and classification as before
    germany_zonal = rs.zonal_stats(
        germany_ea.geometry, str(SMOD_EA), categorical=True, nodata=0,
        all_touched=False  # centre of pixel must fall within polygon
    )
    
    # Aggregate all zones (in case there are multiple polygons)
    total_counts = {}
    for zone in germany_zonal:
        for code, count in zone.items():
            total_counts[code] = total_counts.get(code, 0) + count
    
    # Calculate areas by category
    urban_cells = sum(total_counts.get(c, 0) for c in URBAN_CODES)
    rural_cells = sum(total_counts.get(c, 0) for c in RURAL_CODES) 
    other_cells = sum(total_counts.get(c, 0) for c in OTHER_CODES)
    total_cells = urban_cells + rural_cells + other_cells
    
    # Convert to km²
    urban_area = urban_cells * PIXEL_AREA_KM2
    rural_area = rural_cells * PIXEL_AREA_KM2
    other_area = other_cells * PIXEL_AREA_KM2
    total_raster_area = total_cells * PIXEL_AREA_KM2
    
    # Calculate percentages
    pct_urban = (urban_area / total_raster_area * 100) if total_raster_area > 0 else 0
    pct_rural = (rural_area / total_raster_area * 100) if total_raster_area > 0 else 0
    pct_other = (other_area / total_raster_area * 100) if total_raster_area > 0 else 0
    
    print(f"\n📊 Germany Area Breakdown:")
    print(f"=" * 50)
    print(f"{'Category':<20} {'Area (km²)':<15} {'Percentage':<12} {'Pixels':<10}")
    print(f"{'-'*20} {'-'*15} {'-'*12} {'-'*10}")
    print(f"{'Urban':<20} {urban_area:>13,.0f} {pct_urban:>10.1f}% {urban_cells:>8,}")
    print(f"{'Rural':<20} {rural_area:>13,.0f} {pct_rural:>10.1f}% {rural_cells:>8,}")
    print(f"{'Other':<20} {other_area:>13,.0f} {pct_other:>10.1f}% {other_cells:>8,}")
    print(f"{'-'*20} {'-'*15} {'-'*12} {'-'*10}")
    print(f"{'Total (raster)':<20} {total_raster_area:>13,.0f} {'100.0%':>10} {total_cells:>8,}")
    print(f"{'Total (polygon)':<20} {germany_polygon_area:>13,.0f} {'-':>10} {'-':>8}")
    
    # Calculate difference between polygon and raster areas
    area_diff = total_raster_area - germany_polygon_area
    area_diff_pct = (area_diff / germany_polygon_area * 100) if germany_polygon_area > 0 else 0
    
    print(f"\n🔍 Area Consistency Check:")
    print(f"   Polygon area:     {germany_polygon_area:,.0f} km²")
    print(f"   Raster area:      {total_raster_area:,.0f} km²") 
    print(f"   Difference:       {area_diff:+,.0f} km² ({area_diff_pct:+.1f}%)")
    
    # Create summary DataFrame for export
    germany_summary = pd.DataFrame({
        'category': ['Urban', 'Rural', 'Other', 'Total'],
        'area_km2': [urban_area, rural_area, other_area, total_raster_area],
        'percentage': [pct_urban, pct_rural, pct_other, 100.0],
        'pixel_count': [urban_cells, rural_cells, other_cells, total_cells]
    })
    
    # Export to CSV
    germany_summary.to_csv("germany_total_area_breakdown.csv", index=False)
    print(f"\n💾 Summary exported to: germany_total_area_breakdown.csv")
    
    print(f"\n📋 Key Findings:")
    print(f"   • Germany is predominantly rural: {pct_rural:.1f}% of the country")
    print(f"   • Urban areas cover: {pct_urban:.1f}% ({urban_area:,.0f} km²)")
    print(f"   • This provides context for the healthcare accessibility analysis above")


🇩🇪 Calculating Germany's total area breakdown by urban/rural categories
📦 Loaded Germany boundary from germany-detailed-boundary_917.geojson
📐 Germany polygon area: 357,674 km²
🏙️ Calculating urban/rural breakdown using SMOD raster...

📊 Germany Area Breakdown:
Category             Area (km²)      Percentage   Pixels    
-------------------- --------------- ------------ ----------
Urban                       35,094        9.8%   35,094
Rural                      321,267       89.8%  321,267
Other                        1,325        0.4%    1,325
-------------------- --------------- ------------ ----------
Total (raster)             357,686     100.0%  357,686
Total (polygon)            357,674          -        -

🔍 Area Consistency Check:
   Polygon area:     357,674 km²
   Raster area:      357,686 km²
   Difference:       +12 km² (+0.0%)

💾 Summary exported to: germany_total_area_breakdown.csv

📋 Key Findings:
   • Germany is predominantly rural: 89.8% of the country
   • Urban area

In [ ]:
# ---- 9. Calculate Isochrone Coverage as Percentage of Total Germany ----

print("📊 Calculating isochrone coverage as percentage of Germany's total urban/rural areas")
print("=" * 80)

# Check if we have the required data from previous cells
if 'unions' not in locals():
    print("❌ Missing 'unions' data from isochrone analysis")
    print("   Please run the isochrone coverage analysis cell first")
elif 'germany_summary' not in locals():
    print("❌ Missing 'germany_summary' data from Germany total area calculation")
    print("   Please run the Germany area breakdown cell first")
else:
    # Extract total Germany areas from the summary
    total_germany_urban = germany_summary[germany_summary['category'] == 'Urban']['area_km2'].iloc[0]
    total_germany_rural = germany_summary[germany_summary['category'] == 'Rural']['area_km2'].iloc[0]
    total_germany_other = germany_summary[germany_summary['category'] == 'Other']['area_km2'].iloc[0]
    total_germany_area = germany_summary[germany_summary['category'] == 'Total']['area_km2'].iloc[0]
    
    print(f"🇩🇪 Germany baseline areas:")
    print(f"   Total area:  {total_germany_area:,.0f} km²")
    print(f"   Urban area:  {total_germany_urban:,.0f} km² ({total_germany_urban/total_germany_area*100:.1f}%)")
    print(f"   Rural area:  {total_germany_rural:,.0f} km² ({total_germany_rural/total_germany_area*100:.1f}%)")
    print(f"   Other area:  {total_germany_other:,.0f} km² ({total_germany_other/total_germany_area*100:.1f}%)")
    
    # Calculate coverage percentages for each isochrone
    coverage_results = []
    
    for _, row in unions.iterrows():
        facility_type = row['facility_type']
        time_bin = row['time_bin']
        
        # Coverage areas from isochrones
        iso_urban_area = row['urban_km2']
        iso_rural_area = row['rural_km2']
        iso_other_area = row['other_km2']
        iso_total_area = row['total_km2']
        
        # Calculate percentages of Germany's total areas
        pct_urban_covered = (iso_urban_area / total_germany_urban * 100) if total_germany_urban > 0 else 0
        pct_rural_covered = (iso_rural_area / total_germany_rural * 100) if total_germany_rural > 0 else 0
        pct_other_covered = (iso_other_area / total_germany_other * 100) if total_germany_other > 0 else 0
        pct_total_covered = (iso_total_area / total_germany_area * 100) if total_germany_area > 0 else 0
        
        coverage_results.append({
            'facility_type': facility_type,
            'time_bin': time_bin,
            'urban_coverage_pct': pct_urban_covered,
            'rural_coverage_pct': pct_rural_covered,
            'other_coverage_pct': pct_other_covered,
            'total_coverage_pct': pct_total_covered,
            'urban_area_km2': iso_urban_area,
            'rural_area_km2': iso_rural_area,
            'other_area_km2': iso_other_area,
            'total_area_km2': iso_total_area
        })
    
    # Convert to DataFrame for easier analysis
    coverage_df = pd.DataFrame(coverage_results)
    
    print(f"\n📋 Isochrone Coverage of Germany's Total Areas:")
    print(f"=" * 60)
    
    # Display results organized by facility type
    for facility_type in coverage_df['facility_type'].unique():
        facility_data = coverage_df[coverage_df['facility_type'] == facility_type].sort_values('time_bin')
        
        print(f"\n🏥 {facility_type}:")
        print(f"{'Time':<6} {'Urban %':<8} {'Rural %':<8} {'Other %':<8} {'Total %':<8} {'Urban km²':<10} {'Rural km²':<10} {'Other km²':<10}")
        print(f"{'-'*6} {'-'*8} {'-'*8} {'-'*8} {'-'*10} {'-'*10}")
        
        for _, row in facility_data.iterrows():
            print(f"{row['time_bin']:>4}min {row['urban_coverage_pct']:>6.1f}% "
                  f"{row['rural_coverage_pct']:>6.1f}% {row['total_coverage_pct']:>6.1f}% "
                  f"{row['urban_area_km2']:>8,.0f} {row['rural_area_km2']:>8,.0f}")
    
    # Create summary comparison table
    print(f"\n📊 Coverage Efficiency Analysis:")
    print(f"=" * 50)
    print(f"{'Facility Type':<25} {'Time':<6} {'Urban/Rural Ratio':<18}")
    print(f"{'-'*25} {'-'*6} {'-'*18}")
    
    for _, row in coverage_df.iterrows():
        if row['rural_coverage_pct'] > 0:
            coverage_ratio = row['urban_coverage_pct'] / row['rural_coverage_pct']
        else:
            coverage_ratio = float('inf') if row['urban_coverage_pct'] > 0 else 0
        
        ratio_str = f"{coverage_ratio:.2f}" if coverage_ratio != float('inf') else "∞"
        print(f"{row['facility_type']:<25} {row['time_bin']:>4}min {ratio_str:>15}")
    
    # Export detailed results
    coverage_df.to_csv("isochrone_coverage_of_germany.csv", index=False)
    print(f"\n💾 Detailed coverage data exported to: isochrone_coverage_of_germany.csv")
    
    # Key insights
    print(f"\n💡 Key Coverage Insights:")
    print(f"   🏙️ Urban areas have higher coverage efficiency (smaller areas, better access)")
    print(f"   🌾 Rural coverage percentages show how much of Germany's countryside is served")
    print(f"   ⏱️ Longer time thresholds show diminishing returns in additional coverage")
    
    # Find best and worst coverage scenarios
    max_urban_row = coverage_df.loc[coverage_df['urban_coverage_pct'].idxmax()]
    max_rural_row = coverage_df.loc[coverage_df['rural_coverage_pct'].idxmax()]
    
    print(f"\n🎯 Best Coverage Scenarios:")
    print(f"   Urban: {max_urban_row['facility_type']} at {max_urban_row['time_bin']}min covers "
          f"{max_urban_row['urban_coverage_pct']:.1f}% of Germany's urban areas")
    print(f"   Rural: {max_rural_row['facility_type']} at {max_rural_row['time_bin']}min covers "
          f"{max_rural_row['rural_coverage_pct']:.1f}% of Germany's rural areas")
    


📊 Calculating isochrone coverage as percentage of Germany's total urban/rural areas
🇩🇪 Germany baseline areas:
   Total area:  357,686 km²
   Urban area:  35,094 km² (9.8%)
   Rural area:  321,267 km² (89.8%)
   Other area:  1,325 km² (0.4%)

📋 Isochrone Coverage of Germany's Total Areas:

🏥 CT Hospitals:
Time   Urban %  Rural %  Other %  Total %  Urban km²  Rural km²  Other km² 
------ -------- -------- -------- ---------- ----------
  15min   78.8%   32.1%   36.5%   27,637  103,043
  30min   99.4%   88.6%   89.5%   34,892  284,712
  45min  100.1%   98.4%   98.4%   35,130  316,190
  60min  100.2%   99.1%   99.2%   35,148  318,433

🏥 Extended Stroke Units:
Time   Urban %  Rural %  Other %  Total %  Urban km²  Rural km²  Other km² 
------ -------- -------- -------- ---------- ----------
  15min   49.1%   10.9%   14.6%   17,224   35,135
  30min   92.1%   60.2%   63.2%   32,311  193,531
  45min   98.8%   88.7%   89.6%   34,684  285,012
  60min   99.9%   97.1%   97.2%   35,076  311,941

🏥 